# Imports

In [54]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set random seeds

In [55]:
tf.random.set_seed(42)
np.random.seed(42)

# Load data

In [56]:
raw_df = pd.read_csv("datasets/5_split/df_full.csv")

In [57]:
df = raw_df.copy()

# Separate features

In [58]:
# Dependent variables
labels = df.pop('very_good_health')

# CV folds
fold_ids = df.pop("fold_id_python")
folds = np.unique(fold_ids)

# Independent variables
features = df.drop(columns = ["fold_id_r"])

# Define hyperparameter selection function

In [59]:
def get_hyperparameters():
    no_of_layers = np.random.randint(1, 10)
    no_of_nodes = []
    for i in range(0, no_of_layers):
        no_of_nodes.append(np.random.randint(10, 100))
    learning_rate = np.random.uniform(0.0001, 0.01)
    epochs = np.random.randint(20, 500)
    patience = np.random.randint(5, 30)
    return no_of_layers, no_of_nodes, learning_rate, epochs, patience
    

# Define build model function

In [ ]:
def build_model(train_features, no_of_layers, no_of_nodes, learning_rate):

    layers = []

    normaliser = keras.layers.Normalization(axis = -1)
    normaliser.adapt(np.array(train_features))
    layers.append(normaliser)

    for layer_no in range(no_of_layers):
        layers.append(keras.layers.Dense(no_of_nodes[layer_no], activation = "relu"))

    layers.append(keras.layers.Dense(1))     # Single output for regression value

    model = keras.Sequential(
        layers,
    )

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
        loss='mse'
    )

    return model

# Define early stopper function

In [ ]:
def build_early_stopper(patience):
    early_stopper = keras.callbacks.EarlyStopping(
        monitor = "val_loss",
        patience = patience,
        restore_best_weights = True,
        verbose = 1
    )
    return early_stopper

# Build and evaluate model

In [ ]:
# Initialise evaluation results array
evaluation_results = []

# Cross-validation loop
for fold in folds:
    print(f"\n --- Training on fold {fold} ---")

    # Separate data into training and validation sets
    is_in_validation_set = fold_ids == fold
    is_in_training_set = ~is_in_validation_set

    train_features = features.loc[is_in_training_set]
    train_labels = labels.loc[is_in_training_set]

    val_features = features.loc[is_in_validation_set]
    val_labels = labels.loc[is_in_validation_set]

    no_of_layers, no_of_nodes, learning_rate, epochs, patience = get_hyperparameters()
    
    # Build model
    model = build_model(train_features, no_of_layers, no_of_nodes, learning_rate)

    # Define early stopper inside loop to avoid storing state between folds


    # Fit model
    model.fit(
        train_features,
        train_labels,
        epochs = epochs,
        validation_split = 0.2,
        callbacks = [early_stopper]
    )

    # Get predictions using fitted model
    predictions = model.predict(val_features).flatten()

    # Get accuracy scores
    mae = mean_absolute_error(val_labels, predictions)
    mse = mean_squared_error(val_labels, predictions)
    r2 = r2_score(val_labels, predictions)

    # Add scores for current fold to results
    evaluation_results.append({
        "fold": fold,
        "MAE": mae,
        "MSE": mse,
        "R2": r2
    })


 --- Training on fold 0 ---
Epoch 1/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0198 - val_loss: 0.0087
Epoch 2/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0033 - val_loss: 0.0040
Epoch 3/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0018 - val_loss: 0.0031
Epoch 4/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - val_loss: 0.0028
Epoch 5/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0012 - val_loss: 0.0027
Epoch 6/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.9613e-04 - val_loss: 0.0025
Epoch 7/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.6301e-04 - val_loss: 0.0025
Epoch 8/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.5384e-04 - val_loss: 0.0024
Epoch 9/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.6666e-04 - val_loss: 0.0024
Epoch 10/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.8301e-04 - val_loss: 0.0024
Epoch 11/107
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.1432e-04 - val_l

# Print hyperparameters

# Print results

In [ ]:
evaluation_results